In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from timm import create_model
from torch.cuda.amp import autocast, GradScaler
from PIL import Image
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau

C:\tools\Anaconda\envs\FPY_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = datasets.ImageFolder(root='DEPTH_COMBINED DATASET2/train', transform=transform)
val_dataset = datasets.ImageFolder(root='DEPTH_COMBINED DATASET2/val', transform=transform)
test_dataset = datasets.ImageFolder(root='DEPTH_COMBINED DATASET2/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model = create_model("deit_base_patch16_224", pretrained=True, num_classes=2)
model = model.to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.01) 
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=3, verbose=True)

scaler = GradScaler()

early_stopping_patience = 8  
best_val_acc = 0
patience_counter = 0
best_model_path = "saved_models/4B-2D-Uni-deit.pth"

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs):
    global best_val_acc, patience_counter 
    
    for epoch in range(epochs):
        model.train()
        train_loss, correct = 0, 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
    
        model.eval()
        val_loss, val_correct = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_correct += (outputs.argmax(1) == labels).sum().item()
        
        val_acc = val_correct / len(val_dataset)
        scheduler.step(val_loss / len(val_loader))  # Adjust Learning Rate

        print(f"Epoch {epoch+1}/{epochs}, "
              f"Train Loss: {train_loss/len(train_loader):.4f}, "
              f"Train Acc: {correct/len(train_dataset):.4f}, "
              f"Val Loss: {val_loss/len(val_loader):.4f}, "
              f"Val Acc: {val_acc:.4f}")
        print(f'Current LR: {optimizer.param_groups[0]["lr"]:.6f}')
            

        global best_model_path
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), best_model_path)  # Save best model
            print(f" New Best Model Saved! Validation Accuracy: {val_acc:.4f}")
        else:
            patience_counter += 1
            print(f" No improvement for {patience_counter} epochs...")

        if patience_counter >= early_stopping_patience:
            print(" Early Stopping triggered! Loading best model...")
            model.load_state_dict(torch.load(best_model_path))
            break



train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=30)

model.load_state_dict(torch.load(best_model_path))
print(f" Best model loaded from {best_model_path}")

C:\tools\Anaconda\envs\FPY_env\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
C:\Users\ajeet\AppData\Local\Temp\ipykernel_21164\2208841501.py:7: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
C:\Users\ajeet\AppData\Local\Temp\ipykernel_21164\2208841501.py:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Enable mixed precision training


Epoch 1/30, Train Loss: 0.5519, Train Acc: 0.7096, Val Loss: 0.4674, Val Acc: 0.7681
Current LR: 0.000100
 New Best Model Saved! Validation Accuracy: 0.7681
Epoch 2/30, Train Loss: 0.4242, Train Acc: 0.8021, Val Loss: 0.4115, Val Acc: 0.8013
Current LR: 0.000100
 New Best Model Saved! Validation Accuracy: 0.8013
Epoch 3/30, Train Loss: 0.3799, Train Acc: 0.8294, Val Loss: 0.3728, Val Acc: 0.8334
Current LR: 0.000100
 New Best Model Saved! Validation Accuracy: 0.8334
Epoch 4/30, Train Loss: 0.3445, Train Acc: 0.8453, Val Loss: 0.4213, Val Acc: 0.8203
Current LR: 0.000100
 No improvement for 1 epochs...
Epoch 5/30, Train Loss: 0.3317, Train Acc: 0.8554, Val Loss: 0.3483, Val Acc: 0.8387
Current LR: 0.000100
 New Best Model Saved! Validation Accuracy: 0.8387
Epoch 6/30, Train Loss: 0.3080, Train Acc: 0.8651, Val Loss: 0.3570, Val Acc: 0.8413
Current LR: 0.000100
 New Best Model Saved! Validation Accuracy: 0.8413
Epoch 7/30, Train Loss: 0.2917, Train Acc: 0.8767, Val Loss: 0.3386, Val Acc:

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

def predict_and_display_image(image_path, model, transform, device):
    model.eval()

    image = Image.open(image_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        predicted_class = output.argmax(1).item()

    label = "Stable" if predicted_class == 0 else "Unstable"

    plt.figure(figsize=(6,6))
    plt.imshow(image)
    plt.axis("off")
    plt.title(f"Prediction: {label}", fontsize=14, fontweight='bold')
    plt.show()

    return label



transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = create_model("deit_base_patch16_224", pretrained=False, num_classes=2)
model = model.to(device)


model.load_state_dict(torch.load("saved_models/4B-2D-Non_deit_block_classifier.pth", map_location=device))


model.eval()

image_path = "4B-2D-Non/test/Stable/render_148.png"
prediction = predict_and_display_image(image_path, model, transform, device)
print(f"Prediction: {prediction}")


In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predicted = outputs.argmax(1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds) * 100 
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')

    print(f"Test Accuracy: {accuracy:.2f}%")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    print("\nClassification Report:\n", classification_report(all_labels, all_preds, target_names=["Stable", "Unstable"]))


test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = create_model("deit_base_patch16_224", pretrained=False, num_classes=2)
model = model.to(device)

model.load_state_dict(torch.load("saved_models/4B-2D-Uni-deit.pth", map_location=device))


model.eval()


test_dataset = datasets.ImageFolder(root='DEPTH_COMBINED DATASET2/test', transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

evaluate_model(model, test_loader, device)


C:\Users\ajeet\AppData\Local\Temp\ipykernel_22792\3063416685.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("saved_models/4B-2D-Uni-de

Test Accuracy: 88.59%
Precision: 0.8891
Recall: 0.8819
F1-score: 0.8855

Classification Report:
               precision    recall  f1-score   support

      Stable       0.88      0.89      0.89      1600
    Unstable       0.89      0.88      0.89      1600

    accuracy                           0.89      3200
   macro avg       0.89      0.89      0.89      3200
weighted avg       0.89      0.89      0.89      3200

